In [0]:
import numpy as np
import keras
import keras.utils
from keras import backend as K
from keras import Model
from keras import utils as np_utils
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.layers import Softmax, Dropout,Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/train',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # 
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/valid',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical')

Found 1860 images belonging to 8 classes.
Found 396 images belonging to 8 classes.


In [0]:
from keras import applications
epochs = 50

model = applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(256,256,3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(8, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy",
                    optimizer = optimizers.SGD(lr=0.0001,momentum=0.9),
                    metrics=["accuracy"])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 6s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [0]:
# Train the model 
history = model_final.fit_generator(
train_generator,
epochs = epochs,
steps_per_epoch = 60 ,
validation_data = validation_generator,
validation_steps = 16,
callbacks = [checkpoint, early])

Epoch 1/50
60/60 [==============================] - 3422s 57s/step - loss: 1.4462 - acc: 0.5760 - val_loss: 2.4924 - val_acc: 0.0752

Epoch 00001: val_acc improved from -inf to 0.07520, saving model to vgg19_1.h5
Epoch 2/50
 1/60 [..............................] - ETA: 55:07 - loss: 0.4099 - acc: 0.8750

# ResNet50 Conv Training

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [0]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras import optimizers
from keras.preprocessing import image
from keras.models import load_model

In [0]:
TRAIN_DIR = '/content/drive/My Drive/train'
VALID_DIR = '/content/drive/My Drive/valid'
SIZE = (224, 224)
BATCH_SIZE = 16

In [0]:
model = load_model('/content/drive/My Drive/resnet50_best2(finetune).h5')

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
for layer in model.layers:
    layer.trainable=True


# set_trainable = False
# for layer in model.layers:
#     if layer.name == 'res5a_branch2a':
#       set_trainable = True
#     if set_trainable:
#       layer.trainable = True
#     else:
#       layer.trainable = False
      
model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])      

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
if __name__ == "__main__":
    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

    num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
    num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

    gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,rotation_range=20)
    val_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=False)

    batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)

#     model = load_model('/content/resnet50_final(conv_train).h5')

    classes = list(iter(batches.class_indices))
    
#     for layer in model.layers:
#         layer.trainable=True
    
#     set_trainable = False
#     for layer in model.layers:
#       if layer.name == 'res5c_branch2a':
#         set_trainable = True
#       if set_trainable:
#         layer.trainable = True
#       else:
#         layer.trainable = False
        
    model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(patience=10)
    checkpointer = ModelCheckpoint('resnet50_best(finetune)_rep.h5', verbose=1, save_best_only=True)
    
    model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=50, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)

model.save('resnet50_final(finetune)_rep.h5')    

Found 1860 images belonging to 8 classes.
Found 396 images belonging to 8 classes.
Epoch 1/50
116/116 [==============================] - 60s 520ms/step - loss: 0.3885 - acc: 0.8836 - val_loss: 0.7302 - val_acc: 0.7865

Epoch 00001: val_loss improved from inf to 0.73022, saving model to resnet50_best(finetune)_rep.h5
Epoch 2/50
116/116 [==============================] - 51s 436ms/step - loss: 0.4095 - acc: 0.8761 - val_loss: 0.7355 - val_acc: 0.7921

Epoch 00002: val_loss did not improve from 0.73022
Epoch 3/50
116/116 [==============================] - 53s 454ms/step - loss: 0.3987 - acc: 0.8755 - val_loss: 0.7624 - val_acc: 0.7868

Epoch 00003: val_loss did not improve from 0.73022
Epoch 4/50
116/116 [==============================] - 54s 461ms/step - loss: 0.3761 - acc: 0.8917 - val_loss: 0.7604 - val_acc: 0.7816

Epoch 00004: val_loss did not improve from 0.73022
Epoch 5/50
116/116 [==============================] - 54s 466ms/step - loss: 0.3431 - acc: 0.8965 - val_loss: 0.7340 - va

In [0]:
import cv2
import numpy as np
img = cv2.imread('/content/drive/My Drive/doordent.jpeg')
x = cv2.resize(img, (224, 224))
y = x.reshape(-1, 224, 224, 3) 

In [0]:
y.shape

(1, 224, 224, 3)

In [0]:
from keras.models import load_model
model1 = load_model('/content/drive/My Drive/resnet50_best(conv_train_5c).h5')

In [0]:
model1.predict(y)

array([[4.0535745e-08, 7.3388577e-08, 9.9999082e-01, 4.9541263e-06,
        2.4745805e-06, 1.5010328e-07, 1.3966560e-06, 2.9335561e-09]],
      dtype=float32)

In [0]:
y_prob = model1.predict(y) 
y_classes = y_prob.argmax(axis=-1)
print(y_classes)

[6]


# **NEW**